### **CV Internal Workshop - YOLO**

In [1]:
import os
import json
import shutil
from ultralytics.data.utils import autosplit
from ultralytics import YOLO
from PIL import Image
import cv2
import yaml
from ultralytics import YOLO
import streamlit as st
from PIL import Image
import torch

#### **Data Preparation**

In [2]:
def coco_to_yolo(coco_json_path, data_dir, output_folder):
    with open(coco_json_path, 'r') as f:
        data = json.load(f)

    # Create the FINAL_OUTPUT folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Mapping for class IDs
    class_mapping = {4: 0, 15: 1, 11: 2}  # Mapping for class 4, 15, and 11

    # List to store YOLO annotations for all images
    all_yolo_annotations = []

    for annotation in data['annotations']:
        category_id = annotation['category_id']
        if category_id in [4, 15, 11]:  # Include class 11
            image_info = next(img for img in data['images'] if img['id'] == annotation['image_id'])
            image_path = os.path.join(data_dir, image_info['file_name'])

            # Replace special characters in the filename to prevent errors
            image_filename = os.path.basename(image_info['file_name']).replace('/', '_').replace('\\', '_')

            # Copy the image to the FINAL_OUTPUT folder
            shutil.copy(image_path, os.path.join(output_folder, image_filename))

            image_id = image_info['id']
            image_width = image_info['width']
            image_height = image_info['height']

            annotations = [annotation for annotation in data['annotations'] if annotation['image_id'] == image_id]

            yolo_annotations = []
            for annotation in annotations:
                category_id = annotation['category_id']
                bbox = annotation['bbox']

                # Filter only hardhat, person, and class 11
                if category_id in [4, 15, 11]:
                    # Get bbox coordinates
                    x1, y1, w, h = bbox
                    xc = x1 + (w / 2)
                    yc = y1 + (h / 2)

                    # Determine the new class
                    new_class = class_mapping[category_id]

                    # Normalize coordinates
                    x_center_normalized = xc / image_width
                    y_center_normalized = yc / image_height
                    width_normalized = w / image_width
                    height_normalized = h / image_height

                    # Create YOLO annotation string
                    yolo_annotations.append(f"{new_class} {x_center_normalized} {y_center_normalized} {width_normalized} {height_normalized}")

            # Add YOLO annotations to the list
            all_yolo_annotations.extend(yolo_annotations)

            # Write YOLO annotations to a text file in the FINAL_OUTPUT folder
            output_txt_filename = os.path.splitext(image_filename)[0] + '.txt'
            output_txt_path = os.path.join(output_folder, output_txt_filename)
            with open(output_txt_path, 'w') as out_file:
                out_file.write('\n'.join(yolo_annotations))

    # Write all YOLO annotations to a single text file in the FINAL_OUTPUT folder
    all_yolo_txt_path = os.path.join(output_folder, 'all_yolo_annotations.txt')
    with open(all_yolo_txt_path, 'w') as out_file:
        out_file.write('\n'.join(all_yolo_annotations))

# Example usage
coco_json_path = 'ppe.json'
data_dir = 'data'
output_folder = 'FINAL_OUTPUT'
coco_to_yolo(coco_json_path, data_dir, output_folder)


In [3]:
autosplit(path=output_folder, weights=(0.8, 0.1, 0.1), annotated_only=False)

Autosplitting images from FINAL_OUTPUT


  0%|          | 0/175 [00:00<?, ?it/s]

100%|██████████| 175/175 [00:00<00:00, 422.25it/s]


#### **Model Training**

In [6]:
# Load a model
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='ppe.yaml', epochs=150, imgsz=640)

Ultralytics YOLOv8.2.9  Python-3.10.0 torch-2.3.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=ppe.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

train: Scanning C:\Users\radindafina.radinzul\OneDrive - PETRONAS\Documents\FYP\datasets\FINAL_OUTPUT... 128 images, 0 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 203.47it/s]

train: New cache created: C:\Users\radindafina.radinzul\OneDrive - PETRONAS\Documents\FYP\datasets\FINAL_OUTPUT.cache



val: Scanning C:\Users\radindafina.radinzul\OneDrive - PETRONAS\Documents\FYP\datasets\FINAL_OUTPUT... 22 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<00:00, 130.98it/s]


val: New cache created: C:\Users\radindafina.radinzul\OneDrive - PETRONAS\Documents\FYP\datasets\FINAL_OUTPUT.cache
Plotting labels to runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train8
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    136/150         0G     0.4997       0.33     0.8615        133        640: 100%|██████████| 8/8 [40:46<00:00, 305.79s/it]   
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:19<00:00, 19.17s/it]

                   all         22         98      0.898      0.836       0.89      0.563



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    137/150         0G     0.5705     0.3658     0.8495        162        640:  12%|█▎        | 1/8 [00:28<03:21, 28.76s/it]

#### **Model Evaluation**

In [12]:
metrics = model.val(data='ppe.yaml')

Ultralytics YOLOv8.1.36 🚀 Python-3.11.3 torch-2.2.2+cpu CPU (Intel Core(TM) i7-1065G7 1.30GHz)


val: Scanning C:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT.cache... 22 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:08<00:00,  4.22s/it]


                   all         22         98      0.824      0.904      0.906      0.568
               Hardhat         22         49      0.936       0.89       0.94      0.553
                Person         22         49      0.713      0.918      0.873      0.582
Speed: 2.6ms preprocess, 261.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train33


In [19]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022ABAD86410>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [20]:
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

In [21]:
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("mAP50: {:.4f}".format(mAP50))
print("mAP50-95: {:.4f}".format(mAP50_95))

Precision: 0.9182
Recall: 0.8357
mAP50: 0.9107
mAP50-95: 0.5568


#### **Predictions**

In [18]:
# Load the YAML configuration file
with open('ppe.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
predicts = model.predict(source=config['val'], save=True, save_txt=True)

image 1/22 c:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT\051619e9-1b4c-4efb-bef8-63528a7d17ef.jpg: 448x640 2 Hardhats, 3 Persons, 419.5ms
image 2/22 c:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT\20231103_035117043_iOS.jpg: 480x640 1 Hardhat, 2 Persons, 282.9ms
image 3/22 c:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT\Camera1_01-11-2023_10-16-21_1380.jpg: 576x640 6 Hardhats, 6 Persons, 313.6ms
image 4/22 c:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT\Camera4_23-11-2023_11-46-45_6560.jpg: 384x640 1 Hardhat, 2 Persons, 194.4ms
image 5/22 c:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT\Camera4_23-11-2023_14-49-18_13620.png: 384x640 2 Hardhats, 3 Persons, 206.6ms
image 6/22 c:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT\Camera4_23-11-2023_15-38-47_2000.jpg: 384x640 2 Hardhats, 4 Persons, 182.5ms
image 7/22 c:\Users\radin\Documents\FYP\datasets\FINAL_OUTPUT\Camera4_23-11-2023_15-48-34_8145.png: 384x640 5 Hardhats, 10 Persons, 190.0ms
image 8/22 c:\Users\radin\Documen

In [30]:
model_weights_path = "model.pt"

# Save the model's state_dict to the specified file path
torch.save(model.state_dict(), model_weights_path)


In [36]:
import torch

# Define the file path to the saved model
model_weights_path = "model.pt"

# Load the saved model using torch.load()
saved_model = torch.load(model_weights_path, map_location=torch.device('cpu'))

# Print the keys of the loaded state_dict
# print(saved_model.keys())

# Print other relevant information such as model architecture
# You can inspect the architecture by printing the model or examining its attributes
# For example:
# print(saved_model['model'])  # This assumes that the model architecture is stored in the state_dict

# Optionally, you can also print the state_dict itself to examine its structure
print(saved_model)  # Uncomment this line to print the entire state_dict


OrderedDict([('model.model.0.conv.weight', tensor([[[[ 9.6921e-01,  1.5963e+00, -6.3792e+00],
          [ 4.3526e+00,  4.4789e+00, -5.9548e-01],
          [-4.9576e+00, -5.4716e+00,  6.2029e+00]],

         [[ 3.4745e+00,  3.8152e+00, -5.8975e+00],
          [ 3.5450e+00,  4.3820e+00,  1.7760e-01],
          [-9.5629e+00, -7.4306e+00,  7.3660e+00]],

         [[ 9.7215e-01,  9.5232e-01, -1.9046e+00],
          [ 1.7035e+00,  2.0148e+00, -1.2725e+00],
          [-2.4509e+00, -2.1866e+00,  2.2130e+00]]],


        [[[ 8.6383e-02, -1.3794e+00, -1.1768e+00],
          [-2.0036e+00, -5.1785e+00, -4.5970e+00],
          [-2.5644e+00, -5.4100e+00, -4.8671e+00]],

         [[ 5.2880e-01,  1.2898e+00,  1.1513e+00],
          [ 1.2126e+00,  2.4804e+00,  2.3963e+00],
          [ 1.3559e+00,  2.3205e+00,  2.4831e+00]],

         [[-7.7581e-01, -1.0326e-01, -7.0234e-02],
          [ 8.3851e-01,  2.6499e+00,  2.2365e+00],
          [ 1.4538e+00,  3.0371e+00,  2.8359e+00]]],


        [[[ 2.6947e+00,

In [38]:
# Save the trained model
model.save('trained_model.pt')

# Optionally, you can also save the metrics and predictions
# # Save metrics
# results.metrics.save()

# # Save predictions
# predicts.save()